# Rclone Installation on AppStream (Windows)

This report instructs how to build a Windows image with Rclone installed. Please follow the [Sleap-Windows-Report] for instructions on installing Sleap on Windows Image as well as a more detailed guide through the image building process for Windows.

- [Windows User Instruction](#Windows-User-Instruction)
- [Windows Build Image Instruction](#Windows-Build-Image-Instruction)


## Windows User Instruction

    
### Mount and unmount s3 bucket
    
The difference between using rclone in Linux and Windows is that in Windows, you should *not* create the mount directory beforehand, and the daemon flag is not available so background process is realize by the ampersand. The config file path must be set manually by the flag `--config-file c:\rclone\rclone.conf` or by setting the environment variable `RCLONE_CONFIG`, as done in this demo. The `--no-console` flag can be used to hide the console that runs the rclone command, the user would then have to use `taskkill /F /IM rclone.exe` to kill all rclone instances. 

    # mount s3 bucket in foreground
    rclone mount s3:aind-appstream-data-dev-temporary MOUNT_PATH
    
    # unmount s3 bucket in foreground
    fg
    ctrl + C
    
    # mount s3 bucket in background
    rclone mount s3:aind-appstream-data-dev-temporary MOUNT_PATH --no-console
    
    # unmount all s3 buckets in background
    taskkill /F /IM rclone.exe

## Windows Build Image Instruction

For a more detailed explanation of each phase, see [Sleap-Windows-Report].

### Admin User Install Package Phase

    # Open Command Prompt as Administrator: install choco
    @"%SystemRoot%\System32\WindowsPowerShell\v1.0\powershell.exe" -NoProfile -InputFormat None -ExecutionPolicy Bypass -Command "iex ((New-Object System.Net.WebClient).DownloadString('https://community.chocolatey.org/install.ps1'))" && SET "PATH=%PATH%;%ALLUSERSPROFILE%\chocolatey\bin"
    
    # Open Command Prompt as usual:
    choco install rclone -y
    choco install winfsp -y
    
    # Create file C:\rclone\rclone.conf with contents:
    [s3]
    type = s3
    provider = AWS
    env_auth = true
    region = us-west-2
    profile = appstream_machine_role
    
    # Create empty directory C:\rclone\logs
    mkdir c:\rclone\logs

    # Create C:\rclone\mount.bat file
    @echo off
    set RCLONE_PATH="C:\ProgramData\chocolatey\bin\rclone.exe"
    set MOUNT_SOURCE=s3:aind-appstream-data-dev-temporary
    set MOUNT_DESTINATION="C:\s3-mount-admin-launchparam"
    set LOG_FILE="C:\rclone\logs\sync_files_launchparam.txt"
    set RCLONE_CONFIG="C:\rclone\rclone.conf"

    %RCLONE_PATH% mount %MOUNT_SOURCE% %MOUNT_DESTINATION% --no-console --log-file %LOG_FILE% --config %RCLONE_CONFIG%

    @echo on
    
    # Edit session script config.json as administrator
        # Content in /appstream/SessionScripts/config
        {
      "SessionStart": {
        "executables": [
          {
            "context": "user",
            "filename": "C:\\rclone\\mount_ss.bat,
            "arguments": "",
            "s3LogEnabled": false
          }
        ],
        "waitingTime": 30
      },
      
    # Append C:\rclone\mount.bat to App launch param
    C:\miniconda3\Scripts\activate.bat C:\miniconda3 & C:\rclone\mount.bat
    
**remove log file from launch param**


### Template User Default Settings Phase

    
**Configure DLC:** Launch DeepLabCut once to let it set up some configuration files (e.g. `.keras`, `.matplotlib`). 
    
    (base) conda activate DEEPLABCUT
    (DEEPLABCUT) python -m deeplabcut
    
**Configure Rclone:**
    setx RCLONE_CONFIG C:\rclone\rclone.conf
    
    # to kill all instances of rclone, specifically the shortcut startup instance and its log filety
    taskkill /F /IM rclone.exe
    
Once completed, click switch back to Admin user in Image Assistance. 

**Check task scheduler**
**Try session script**
**Try laucnh params in image assistance (see powershell example)**

### Test User Testing Phase

```

```

### Optional: Transfer files between S3 buckets with Rclone. 

To test the performance of Sleap on AppStream instances, you may wish to analyze some large video files on S3. During this demo, I used Rclone to transfer a large video file to our appstream bucket for testing. 

**Transfer S3 data between buckets with 2 accounts**

In rclone.conf file, enter two AWS credentials with names `remoteprod` and `remotetest`. `remoteprod` credentials have read access to the video file, and `remotetest` credentials have write access to the appstream bucket. 

    git clone https://github.com/rclone/rclone.git
    cd rclone
    go build
    
    ./rclone copy "remoteprod:aind-behavior-data/pose_estimation_training/DLC annotation/1051155866_524760_20200917.behavior-Shailaja-2023-06-19/videos/1051155866_524760_20200917.behavior.mp4" "remotetest:aind-appstream-data-dev-temporary" --config rclone.conf

    ./rclone ls remotetest:aind-appstream-data-dev-temporary --config rclone.conf

    ./rclone ls "remoteprod:aind-behavior-data/pose_estimation_training/DLC annotation/1051155866_524760_20200917.behavior-Shailaja-2023-06-19/videos" --config rclone.conf